In [3]:
import numpy as np

# sum of two arrays
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
c = a + b
print(c)

[5 7 9]


In [1]:
# from IPython.display import display, HTML

# # Youtube
# YouTubeVideo_ID = 'dzHYjDuRYzs'

# # Adjust the width and height values
# width = 1280  
# height = 720  

# # create a HTML string to center the video
# html_str = """
# <div style="display: flex; justify-content: center;">
#     <iframe width="{}" height="{}" src="https://www.youtube.com/embed/{}" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>
# </div>
# """.format(width, height, YouTubeVideo_ID)

# # Display HTML
# display(HTML(html_str))


In [4]:
# %pip install mysql-connector-python
# # !pip install opencv-python
# %pip install easyocr
# %pip install opencv-contrib-python
# %conda install mathplotlib


In [2]:
import warnings
warnings.filterwarnings('ignore')
# %pip install opencv-python
# %pip install pytesseract
# %pip install easyocr
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import requests
import pytesseract
import easyocr
import re
import mysql.connector
import pytesseract
from PIL import Image
from collections import deque
from mysql.connector import Error

In [3]:
# Database Connection Constants
DB_HOST = 'localhost'
DB_USER = 'root'
DB_PASSWORD = ''
DB_NAME = 'traffic_violations_db'

In [4]:
def detect_traffic_light_color(image, rect):
    # Extract rectangle dimensions
    x, y, w, h = rect
    # Extract region of interest (ROI) from the image based on the rectangle
    roi = image[y:y+h, x:x+w]
    
    # Convert ROI to HSV color space
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Define HSV range for red color
    red_lower = np.array([0, 120, 70])
    red_upper = np.array([10, 255, 255])
    
    # Define HSV range for yellow color
    yellow_lower = np.array([20, 100, 100])
    yellow_upper = np.array([30, 255, 255])

    # Create binary masks for detecting red and yellow in the ROI
    red_mask = cv2.inRange(hsv, red_lower, red_upper)
    yellow_mask = cv2.inRange(hsv, yellow_lower, yellow_upper)
    
    # Font details for overlaying text on the image
    font = cv2.FONT_HERSHEY_TRIPLEX
    font_scale = 1  
    font_thickness = 2  
    
    # Check which color is present based on the masks
    if cv2.countNonZero(red_mask) > 0:
        text_color = (0, 0, 255)
        message = "Detected Signal Status: Stop"
        color = 'red'
    elif cv2.countNonZero(yellow_mask) > 0:
        text_color = (0, 255, 255)
        message = "Detected Signal Status: Caution"
        color = 'yellow'
    else:
        text_color = (0, 255, 0)
        message = "Detected Signal Status: Go"
        color = 'green'
        
    # Overlay the detected traffic light status on the main image
    cv2.putText(image, message, (15, 70), font, font_scale+0.5, text_color, font_thickness+1, cv2.LINE_AA)
    # Add a separator line
    cv2.putText(image, 34*'-', (10, 115), font, font_scale, (255,255,255), font_thickness, cv2.LINE_AA)
    
    # Return the modified image and detected color
    return image, color

In [5]:
class LineDetector:
    def __init__(self, num_frames_avg=10):
        # Initialize two deque queues to hold y-coordinate values across frames
        self.y_start_queue = deque(maxlen=num_frames_avg)
        self.y_end_queue = deque(maxlen=num_frames_avg)

    
    def detect_white_line(self, frame, color, 
                          slope1=0.03, intercept1=920, slope2=0.03, intercept2=770, slope3=-0.8, intercept3=2420):
        
        # Function to map color names to BGR values
        def get_color_code(color_name):
            color_codes = {
                'red': (0, 0, 255),
                'green': (0, 255, 0),
                'yellow': (0, 255, 255)
                 }
            return color_codes.get(color_name.lower())

        frame_org = frame.copy()
        
        # Line equations for defining region of interest (ROI)
        def line1(x): return slope1 * x + intercept1
        def line2(x): return slope2 * x + intercept2
        def line3(x): return slope3 * x + intercept3

        height, width, _ = frame.shape
        
        # Create a mask to spotlight the line's desired area
        mask1 = frame.copy()
        # Set pixels below the first line to black in mask1
        for x in range(width):
            y_line = line1(x)
            mask1[int(y_line):, x] = 0

        mask2 = mask1.copy()
        # Set pixels above the second line to black in mask2
        for x in range(width):
            y_line = line2(x)
            mask2[:int(y_line), x] = 0

        mask3 = mask2.copy()
        # Set pixels to the left of the third line to black in mask3 (final mask)
        for y in range(height):
            x_line = line3(y)
            mask3[y, :int(x_line)] = 0

        # Convert the mask to grayscale
        gray = cv2.cvtColor(mask3, cv2.COLOR_BGR2GRAY)

        # Apply a Gaussian filter to the ROI
        blurred_gray = cv2.GaussianBlur(gray, (7, 7), 0)

        # Apply CLAHE to equalize the histogram
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        gray_clahe = clahe.apply(blurred_gray)

        # Perform edge detection
        edges = cv2.Canny(gray, 30, 100)

        # Perform a dilation and erosion to close gaps in between object edges
        dilated_edges = cv2.dilate(edges, None, iterations=1)
        edges = cv2.erode(dilated_edges, None, iterations=1)

        # Perform Hough Line Transform
        lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=160, maxLineGap=5)

        # Calculate x coordinates for the start and end of the line
        x_start = 0
        x_end = width - 1

        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]

                # Calculate line parameters
                slope = (y2 - y1) / (x2 - x1 + np.finfo(float).eps)  # Add a small number to avoid division by zero
                intercept = y1 - slope * x1

                # Calculate corresponding y coordinates
                y_start = int(slope * x_start + intercept)
                y_end = int(slope * x_end + intercept)

                # Add the y_start and y_end values to the queues
                self.y_start_queue.append(y_start)
                self.y_end_queue.append(y_end)

        # Compute the average y_start and y_end values
        avg_y_start = int(sum(self.y_start_queue) / len(self.y_start_queue)) if self.y_start_queue else 0
        avg_y_end = int(sum(self.y_end_queue) / len(self.y_end_queue)) if self.y_end_queue else 0

        
        # Draw the line
        line_start_ratio=0.32
        x_start_adj = x_start + int(line_start_ratio * (x_end - x_start))  # Adjusted x_start
        avg_y_start_adj = avg_y_start + int(line_start_ratio * (avg_y_end - avg_y_start))  # Adjusted avg_y_start

        # Create a mask with the same size as the frame and all zeros (black)
        mask = np.zeros_like(frame)

        # Draw the line on the mask
        cv2.line(mask, (x_start_adj, avg_y_start_adj), (x_end, avg_y_end), (255, 255, 255), 4)

        # Determine which color channel(s) to change based on the color argument
        color_code = get_color_code(color)
        if color_code == (0, 255, 0):  # Green
            channel_indices = [1]
        elif color_code == (0, 0, 255):  # Red
            channel_indices = [2]
        elif color_code == (0, 255, 255):  # Yellow
            # Yellow in BGR is a combination of green and red channels.
            # Here we modify both green and red channels.
            channel_indices = [1, 2]
        else:
            raise ValueError('Unsupported color')

        # Change the specified color channels of the frame where the mask is white
        for channel_index in channel_indices:
            frame[mask[:,:,channel_index] == 255, channel_index] = 255
                
                
        # Calculate slope and intercept for the average green line
        slope_avg = (avg_y_end - avg_y_start) / (x_end - x_start + np.finfo(float).eps)
        intercept_avg = avg_y_start - slope_avg * x_start

        # Create a mask with the pixels above the green line set to black
        mask_line = np.copy(frame_org)
        for x in range(width):
            y_line = slope_avg * x + intercept_avg - 35
            mask_line[:int(y_line), x] = 0  # set pixels above the line to black

        return frame, mask_line

In [7]:
def apply_ocr_to_image(license_plate_image):    
    # Threshold the image
    _, img = cv2.threshold(license_plate_image, 120, 255, cv2.THRESH_BINARY)

    # Convert OpenCV image format to PIL Image format for pytesseract
    pil_img = Image.fromarray(img)

    # Use pytesseract to extract text from the image
    full_text = pytesseract.image_to_string(pil_img, config='--psm 6')

    return full_text.strip()  # Removing any extra white spaces from the ends

In [8]:
def draw_penalized_text(frame):
    # Set font, scale, thickness, and color
    font = cv2.FONT_HERSHEY_TRIPLEX
    font_scale = 1  
    font_thickness = 2
    color = (255, 255, 255)  # White color
    
    # Initial position for Y-coordinate
    y_pos = 180
    
    # Put title on the frame
    cv2.putText(frame, 'Fined license plates:', (25, y_pos), font, font_scale, color, font_thickness)
    
    # Update Y-coordinate position
    y_pos += 80

    # Loop through all fined license plates
    for text in penalized_texts:
        # Add fined license plate text on the frame
        cv2.putText(frame, '->  '+text, (40, y_pos), font, font_scale, color, font_thickness)
        
        # Update Y-coordinate for next license plate
        y_pos += 60

In [9]:
def create_database_and_table(host, user, password, database):
    try:
        # Create a connection
        connection = mysql.connector.connect(
            host = host,
            user = user,
            password = password
        )
        
        if connection.is_connected():
            # Create a new database cursor
            cursor = connection.cursor()

            # Create a new database using the provided name
            cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database}")
            print(f"Database {database} created successfully!")

            # Use the newly created database
            cursor.execute(f"USE {database}")

            # Create a new table
            cursor.execute("""
                CREATE TABLE IF NOT EXISTS license_plates (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    plate_number VARCHAR(255) NOT NULL UNIQUE,
                    violation_count INT DEFAULT 1
                )
            """)
            print("Table created successfully!")

            cursor.close()

    except Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection.is_connected():
            connection.close()

In [10]:
def update_database_with_violation(plate_number, host, user, password, database):
    try:
        connection = mysql.connector.connect(
            host = host,
            user = user,
            password = password,
            database = database
        )
        
        if connection.is_connected():
            cursor = connection.cursor()

            # Check if the license plate already exists in the table
            cursor.execute(f"SELECT violation_count FROM license_plates WHERE plate_number='{plate_number}'")
            result = cursor.fetchone()
            
            if result:
                # Increment violation_count by 1 if plate_number already exists
                cursor.execute(f"UPDATE license_plates SET violation_count=violation_count+1 WHERE plate_number='{plate_number}'")
            else:
                # Insert a new record if plate_number doesn't exist
                cursor.execute(f"INSERT INTO license_plates (plate_number) VALUES ('{plate_number}')")
            
            connection.commit()
            cursor.close()

    except Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection.is_connected():
            connection.close()

In [11]:
def print_all_violations(host, user, password, database):
    try:
        connection = mysql.connector.connect(
            host = host,
            user = user,
            password = password,
            database = database
        )
        
        if connection.is_connected():
            cursor = connection.cursor()

            # Fetch all violations from the database
            cursor.execute("SELECT plate_number, violation_count FROM license_plates ORDER BY violation_count DESC")
            result = cursor.fetchall()
            
            print("\n")
            print("-"*66)
            print("\nAll Registered Traffic Violations in the Database:\n")
            for record in result:
                print(f"Plate Number: {record[0]}, Violations: {record[1]}")
            
            cursor.close()

    except Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection.is_connected():
            connection.close()

<div style="border-radius:10px; padding: 15px; background-color: #a3b8c7; font-size:115%; text-align:left"> 
    
__4. Clearing Recorded Data ( `clear_license_plates` ):__
- Deletes all the records in the `license_plates` table, effectively resetting the database.
- This function is also optional and can be seen as a tool for data management or debugging purposes. Routine operations might not require its use.

In [12]:
def clear_license_plates(host, user, password, database):
    try:
        connection = mysql.connector.connect(
            host = host,
            user = user,
            password = password,
            database = database
        )
        
        if connection.is_connected():
            cursor = connection.cursor()

            # Delete all records from the table
            cursor.execute("DELETE FROM license_plates")

            connection.commit()
            cursor.close()

    except Error as e:
        print("Error while connecting to MySQL", e)

    finally:
        if connection.is_connected():
            connection.close()

<a id="step8"></a>
# <p style="background-color:#10191f; font-family:calibri; color:white; font-size:150%; text-align:center; border-radius:15px 50px;">Step 8 | Traffic Violation Monitoring Execution</p>

⬆️ [Tabel of Contents](#contents_tabel)

<div style="border-radius:10px; padding: 15px; background-color: #a3b8c7; font-size:115%; text-align:left"> 
    
Finally, I am going to define the `main` function which encapsulates the whole process of detecting license plate violations in a video feed. The function operates on an offline video of traffic, identifies the license plate of vehicles crossing the line when the light is red, and then processes the violations. Here's a comprehensive breakdown of its operations:

__1. Database Setup__: As the function begins, it ensures that the appropriate database and table for storing license plate violations are available.

    
__2. Video Feed__: The function reads the offline video `traffic_video.mp4`, frame by frame, and processes each frame to detect any traffic violations.

    
__3. Traffic Light Detection__: The color of a traffic light for the given frame is detected using the `detect_traffic_light_color` function.

    
__4. White Line Detection__: The function calls upon the `LineDetector` class to pinpoint the white stop line on the road, highlighting it based on the detected traffic light color.

    
__5. Violation Detection & Processing__: If the detected color is red, the program uses the `extract_license_plate` and `apply_ocr_to_image` functions to identify vehicles crossing the white line and then to extract and read their license plates. Due to potential low-quality images fed to the `apply_ocr_to_image` function, some license plate readings may be incorrect. To counteract this, plates are verified against a predefined format, ensuring they match the standard format of a car license plate in the video. Only those that match and haven't been previously logged are recorded as violations.

    
__6. Database Update__: The detected and processed license plates are recorded in the database. If a plate had a previous violation, its count is incremented.

    
__7. Display Updates__: The `draw_penalized_text` function is called, which displays license plates that have been penalized in order on the frame.

    
__8. Termination__: Processing continues until the video feed concludes or until the user terminates with the __ESC__ key. The video is then released, and all GUI windows close.

    
__9. Report Generation__: Finally, the function fetches and lists all violations from the database, providing an overview of all infringements during the video's duration.
____
    
With this approach, the `main` function offers a robust and comprehensive solution to monitor and record traffic violations from a video source, marking its significance in traffic law enforcement and analytics.


In [13]:
def main():
    # Ensure the database and table exist
    create_database_and_table(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)

    # Clear the license plates from the previous run. (Comment out this line if desired!)
    clear_license_plates(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)

    # Open the video file
    vid = cv2.VideoCapture('./traffic_video_modified.mp4')
    # confirm if the video is opened
    if not vid.isOpened():
        raise ValueError("Unable to open video file")
    
    
    # Create detector object
    detector = LineDetector()

    # Loop through each frame in the video
    while True:
        # Read frame
        ret, frame = vid.read()
        
        # Break if frame is not returned
        if not ret:
            break

        # Assuming rect is the rectangle where the traffic light is located
        rect = (1700, 40, 100, 250) 
        
        # Detect traffic light color
        frame, color = detect_traffic_light_color(frame, rect)
        
        # Detect white line
        frame, mask_line = detector.detect_white_line(frame, color)
        
        # Process the frame if the light is red
        if color == 'red':
            # Extract license plate
            frame, license_plate_images = extract_license_plate(frame, mask_line)
            
            # Process each detected license plate
            for license_plate_image in license_plate_images:
                # Apply OCR to the license plate image
                text = apply_ocr_to_image(license_plate_image)
                
                # Add the detected license plate to the list if it matches the pattern and is not already in the list
                if text is not None and re.match("^[A-Z]{2}\s[0-9]{3,4}$", text) and text not in penalized_texts:
                    penalized_texts.append(text)
                    print(f"\nFined license plate: {text}")

                    # Plot the license plate image
                    plt.figure()
                    plt.imshow(license_plate_image, cmap='gray')
                    plt.axis('off')
                    plt.show()
                    
                    # Update the database with the license plate violation
                    update_database_with_violation(text, DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)
        
        # Draw the penalized text onto the frame if there is any
        if penalized_texts:
            draw_penalized_text(frame)

        # Display the frame 
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        # plt.show()

        # Break if ESC key is pressed (uncomment the following line when running on a local system with GUI support)
        # if cv2.waitKey(1) == 27:
        #     break

    # Release the video
    vid.release()
    
    # Close all OpenCV windows (uncomment the following line when running on a local system with GUI support)
    cv2.destroyAllWindows()

    # Print all the violations from the database
    print_all_violations(DB_HOST, DB_USER, DB_PASSWORD, DB_NAME)

<div style="border-radius:10px; padding: 15px; background-color: #a3b8c7; font-size:115%; text-align:left"> 
 
Before diving into the primary operations of our program, a few preparatory steps are crucial:

* __Haar Cascade Initialization and Download__: At the heart of our license plate detection mechanism, we use a Haar cascade classifier. For the Kaggle environment, we first download our pre-trained model, `haarcascade_russian_plate_number.xml`, from the GitHub repository. This model, tailor-made for detecting license plates, is paramount for identifying license plates in our video frames. By initializing the Haar cascade object here, we ensure our program doesn't suffer repeated delays due to multiple initializations.

    
* __Creation of List__: Following this, we initialize a list named `penalized_texts`. This list functions as a repository, capturing the texts of license plates involved in traffic violations. The global definition of this list ensures both the `main` function and the `draw_penalized_text` function can access and update its contents without a hitch.


In [14]:
%%capture
# Download the trained Haar Cascade from the GitHub repository
url = "https://raw.githubusercontent.com/FarzadNekouee/Traffic-Violation-Detection/master/haarcascade_russian_plate_number.xml"
response = requests.get(url)

with open('haarcascade_russian_plate_number.xml', 'wb') as file:
    file.write(response.content)

# Load the trained Haar Cascade
license_plate_cascade = cv2.CascadeClassifier('haarcascade_russian_plate_number.xml')

# Create a list to store unique penalized license plate texts
penalized_texts = []

<div style="border-radius:10px; padding: 15px; background-color: #a3b8c7; font-size:115%; text-align:left"> 

When we start the program, we use a specific line to kick things off:

__Launching the Main Program__: Encapsulated under the familiar Python construct `if __name__ == "__main__":`, this mechanism ensures that our `main` function springs into action only when our script is executed as a standalone entity, and not when it's imported as a module in another script. Once initiated, the `main` function commences its elaborate process – it delves into the video, discerns license plate violations, and meticulously logs any transgressions.
____
    
Employing this precise structure, our code remains modular and systematic, ensuring that each part runs in its rightful context, preserving the integrity and efficiency of our program.

In [15]:
if __name__ == "__main__":
    main()

Database traffic_violations_db created successfully!
Table created successfully!


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

Error in callback <function flush_figures at 0x000001655CDF8180> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

<div style="border-radius:10px; padding: 15px; background-color: #a3b8c7; font-size:115%; text-align:left"> 
    
<h2 align="left"><font color=#10191f>🔔 Instructions for Running the Code:</font></h2>

1. **Local Execution:** 
   - This notebook is set up to showcase the processing steps. To fully execute and view the results on your machine, download this notebook.
   - Ensure you have the necessary dependencies installed and a local MySQL database set up.
   - Replace the placeholders `your_username` and `your_password` in the database connection constants with your own database credentials, as indicated in the previous instruction note.
   - Uncomment the `if __name__ == "__main__":` block to allow execution of the main function.

    
2. **Output Preview:** 
   - In this notebook, immediately following this instruction section, we present a snapshot of the expected results, including detected license plates and logs. 👇
   - For a dynamic preview, check out the embedded YouTube video at the beginning of this notebook which showcases the full output.

    
3. **Full Project on GitHub:** 
   - If you're interested in diving deeper, exploring more features, or contributing, the full project repository is available on [GitHub](https://github.com/FarzadNekouee/Traffic-Violation-Detection). Click on the link to directly access the repo.
   - We encourage you to clone it and experiment locally.
   - Make sure to check the README in the GitHub repository for detailed instructions on setup and execution.


<h3 align="left"><font color=#10191f>💡 Tip:</font></h3>
    
By running this project on your local machine, you can see real-time detections and database updates, giving you a comprehensive understanding of its functionality.


<h3 align="left"><font color=#10191f>Results snapshot:</font></h3>

![Expected Detection Outcomes: A Snapshot](https://raw.githubusercontent.com/FarzadNekouee/Traffic-Violation-Detection/master/result_snapshot.png)


<div style="display: flex; align-items: center; justify-content: center; border-radius: 10px; padding: 20px; background-color: #a3b8c7; font-size: 120%; text-align: center">

<strong>🎯 Feel free to visit the project repository on <a href="https://github.com/FarzadNekouee/Traffic-Violation-Detection">GitHub</a> 🎯</strong>
</div>

<h2 align="left"><font color=#10191f>Best Regards!</font></h2>